In [1]:
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import random
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException, StaleElementReferenceException

In [2]:
# Đọc danh sách link từ file text

with open('film_urls.txt', 'r') as file:
    links1 = [line.strip() for line in file.readlines()]

# Kết hợp hai danh sách links
all_links = links1

In [3]:
# Tùy chọn số lượng link được cào
links = all_links[248:]  # Lấy số lượng link theo yêu cầu

In [4]:
# Tạo danh sách để lưu kết quả
data = []

In [5]:
# Cấu hình Selenium
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Danh sách các User-Agent phổ biến
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
]

# Chọn ngẫu nhiên 1 User-Agent từ danh sách
options.add_argument(f"user-agent={random.choice(user_agents)}")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 5)

# URL cần crawl
url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=1000,&languages=en"
driver.get(url)

In [6]:
def scrape_reviews(link):
    try:
        driver.get(link)
        time.sleep(3)  # Đợi trang tải
        
        # Lấy tên phim
        try:
            movie_title = driver.find_element(By.CSS_SELECTOR, 'div > section > section > div:nth-child(4) > section > section > div.sc-b8cc654b-4.fpCWu > hgroup > h2').text
        except NoSuchElementException:
            movie_title = ""
        
        # Lấy danh sách các bài review
        reviews = driver.find_elements(By.CSS_SELECTOR, 'article.sc-f53ace6f-1.cHwTOl.user-review-item')
        print(len(reviews))
        
        for index, review in enumerate(reviews[:10]):  # Lấy tối đa 100 bài
            # Scroll to the review to ensure it's fully loaded
            stale_attempts = 0
            while stale_attempts < 3:  # Thử lại tối đa 3 lần
                try:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", review)
                    time.sleep(random.uniform(1, 2))  # Nghỉ ngẫу nhiên để tránh bị phát hiện là bot
                    break  # Nếu không có lỗi, thoát khỏi vòng lặp
                except StaleElementReferenceException:
                    print(f"Lỗi StaleElementReferenceException, thử lại lần {stale_attempts + 1}")
                    stale_attempts += 1
                    # Làm mới danh sách phần tử và thử lại
                    reviews = driver.find_elements(By.CSS_SELECTOR, 'article.sc-f53ace6f-1.cHwTOl.user-review-item')
                    review = reviews[index]
            
            # Khởi tạo các biến với giá trị rỗng
            reviewer_name = ""
            rating = ""
            review_text = ""
            likes = ""
            dislikes = ""
            
            # Lấy thông tin review với xử lý ngoại lệ
            try:
                reviewer_name = review.find_element(By.CSS_SELECTOR, 'div.sc-f6867cfd-1.iHZNcU > ul > li:nth-child(1) > a').text
            except NoSuchElementException:
                pass
            
            try:
                rating = review.find_element(By.CSS_SELECTOR, 'div.sc-a2ac93e5-4.gyibOi > span > span.ipc-rating-star--rating').text
            except NoSuchElementException:
                pass
            
            try:
                likes = review.find_element(By.CSS_SELECTOR, 'div.sc-a2ac93e5-8.cxqvuH > div > span:nth-child(2) > span.ipc-voting__label__count.ipc-voting__label__count--up').text
            except NoSuchElementException:
                pass
            
            try:
                dislikes = review.find_element(By.CSS_SELECTOR, 'div.sc-a2ac93e5-8.cxqvuH > div > span:nth-child(4) > span').text
            except NoSuchElementException:
                pass
            
            # Xử lý nội dung review, ưu tiên lấy nếu có nút spoiler
            try:
                # Thử click nút spoiler
                spoiler_button = review.find_element(By.CSS_SELECTOR, 'div.ipc-list-card--border-speech.ipc-list-card--hasActions.ipc-list-card--base.ipc-list-card.sc-a2ac93e5-0.claXGn > div.ipc-list-card__content > button')
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", spoiler_button)
                time.sleep(1)
                spoiler_button.click()
                time.sleep(random.uniform(1, 2))
                
                # Nếu có nút spoiler, lấy nội dung review từ selector mới
                try:
                    review_text = driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section > div > section > div > div.sc-65d2a03-1.hLElui.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.ipc-page-section--sp-pageMargin > article:nth-child(5) > div.ipc-list-card--border-speech.ipc-list-card--hasActions.ipc-list-card--base.ipc-list-card.sc-a2ac93e5-0.claXGn > div.ipc-list-card__content > div:nth-child(3) > div.ipc-html-content.ipc-html-content--base.review-content > div').text
                except NoSuchElementException:
                    pass
            except NoSuchElementException:
                # Nếu không có nút spoiler, lấy nội dung review bình thường
                try:
                    review_text = review.find_element(By.CSS_SELECTOR, 'div.ipc-overflowText.ipc-overflowText--listCard.ipc-overflowText--base > div > div > div').text
                except NoSuchElementException:
                    pass
            
            # Kiểm tra trùng lặp trước khi thêm vào danh sách
            is_duplicate = False
            for entry in data:
                if entry["Reviewer"] == reviewer_name and entry["Review"] == review_text:
                    is_duplicate = True
                    break

            # Nếu không trùng lặp, thêm vào danh sách
            if not is_duplicate:
                data.append({
                    "Movie": movie_title,
                    "Reviewer": reviewer_name,
                    "Rating": int(rating) if rating and rating.isdigit() else "",
                    "Review": review_text,
                    "Likes": int(likes) if likes and likes.isdigit() else "",
                    "Dislikes": int(dislikes) if dislikes and dislikes.isdigit() else "",
                            })

    except Exception as e:
        print(f"Error scraping {link}: {e}")

In [7]:
# Lặp qua từng link
for link in links:
    scrape_reviews(link)

25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
22
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
24
25
25
25
25
19
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
25
16
25
25
25
25
25
25
25
22
25
25
25
25
25
24
25
25
25
24
25
25
25
25
25
25
25
25
25
25
24
25
25
25
25
25
24
25
25
25
25
25
15
23
25
25
25
25
24
25
25
25
25
25
25
25
2

In [8]:
driver.quit()

In [9]:
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)  # Hiển thị đầy đủ nội dung của các ô
df.head()

Movie        Reviewer Rating  \
0  12 Angry Men       ElJimbooo     10   
1  12 Angry Men     henriquelrf     10   
2  12 Angry Men        vukodlak     10   
3  12 Angry Men  CubsandCulture     10   
4  12 Angry Men     navidveradi     10   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13391 entries, 0 to 13390
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Movie     13391 non-null  object
 1   Reviewer  13391 non-null  object
 2   Rating    13391 non-null  object
 3   Review    13391 non-null  object
 4   Likes     13391 non-null  object
 5   Dislikes  13391 non-null  object
dtypes: object(6)
memory usage: 627.8+ KB


In [11]:
df.to_excel('movie_reviews2.xlsx', index=False)

In [12]:
print(df['Reviewer'].value_counts()[df['Reviewer'].value_counts() >= 2])

Reviewer
TheLittleSongbird    159
SnoopyStyle          140
Sleepin_Dragon        88
claudio_carvalho      74
planktonrules         71
                    ... 
nature_whisper         2
Alanjackd              2
Zzacarias              2
danew13                2
LansTaylor             2
Name: count, Length: 1260, dtype: int64


In [13]:
df['Movie'].nunique()

1374